In [1]:
from sdRDM import DataModel
from sdRDM.generator import generate_python_api
import numpy as np
from datetime import datetime

In [2]:
# generate_python_api("specifications/new_calibration.md", "", "CaliPytion")

In [3]:
%reload_ext autoreload
%autoreload 2
from CaliPytion import Calibrator, Standard
from CaliPytion.modified import Sample

In [4]:
def random(xs):
    return xs+np.random.normal(0, 0.03, 1)


PARAM_A = 0.014

CONC = np.linspace(0, 200, 11)
ABSO = CONC*PARAM_A

In [5]:
standard = Standard(
    species_id="s0",
    name="ABTS",
    wavelength=340,
    ph=2,
    temperature=25,
    temperature_unit="C",
    created=datetime.now(),
)

# Add samples
for conc, abso in zip(CONC, ABSO):
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )

In [6]:
calibrator = Calibrator.from_standard(standard, cutoff=1)

In [7]:
calibrator.concentrations

[0.0, 0.0, 0.0, 20.0, 20.0, 20.0, 40.0, 40.0, 40.0, 60.0, 60.0, 60.0]

In [8]:

# Add models to calibrator
linear = calibrator.add_to_models(
    name="linear", equation="a * concentration = signal")
quadratic = calibrator.add_to_models(
    name="quadratic", equation="a * concentration**2 + b * concentration = signal")

print(calibrator)

Calibrator
├── id = calibrator0
├── standard
│   └── Standard
│       ├── id = standard1
│       ├── species_id = s0
│       ├── name = ABTS
│       ├── wavelength = 340.0
│       ├── samples
│       │   ├── 0
│       │   │   └── Sample
│       │   │       ├── id = sample0
│       │   │       ├── concentration = 0.0
│       │   │       ├── conc_unit = umol / l
│       │   │       └── signal = 0.03998664610119449
│       │   ├── 1
│       │   │   └── Sample
│       │   │       ├── id = sample1
│       │   │       ├── concentration = 0.0
│       │   │       ├── conc_unit = umol / l
│       │   │       └── signal = 0.029908585548001833
│       │   ├── 2
│       │   │   └── Sample
│       │   │       ├── id = sample2
│       │   │       ├── concentration = 0.0
│       │   │       ├── conc_unit = umol / l
│       │   │       └── signal = 0.014567241538710035
│       │   ├── 3
│       │   │   └── Sample
│       │   │       ├── id = sample3
│       │   │       ├── concentration = 20.0
│      

In [9]:
calibrator.fit_models()

,AIC,R squared,RMSD
Model Name,,,
linear,-79,0.9867,0.0347
quadratic,-77,0.9868,0.0346


In [10]:
calibrator.fit_statistics

,AIC,R squared,RMSD
Model Name,,,
linear,-79,0.9867,0.0347
quadratic,-77,0.9868,0.0346


In [11]:
print(calibrator.models[0])

Model
├── id = model1
├── name = linear
├── equation = a * concentration = signal
├── parameters
│   └── 0
│       └── Parameter
│           ├── id = parameter0
│           ├── name = a
│           ├── value = 0.013845677157142999
│           ├── init_value = 0.1
│           ├── standard_error = 0.0002798150340324915
│           ├── lower_bound = -inf
│           └── upper_bound = inf
├── aic = -78.64768891881754
├── bic = -78.16278226902953
├── r2 = 0.9867101011373692
├── residuals = [-0.03998664610119449, -0.029908585548001833, -0.014567241538710035, -0.011090099300591683, 0.001565074256186294, ...]
└── rmsd = 0.03472413253603766



In [25]:
calibrator.visualize(linear)

## concentration calculation

In [ ]:
linear._params

{'a': 0.014188716490768596}

In [ ]:
model.calculate_concentrations(
    signals=[1, 1.1, 2.21],
    cutoff=3,
    calibration_range=calibrator.concentrations
)

NameError: name 'model' is not defined

In [27]:
print(calibrator.save_fitted_model(linear))

Standard
├── id = standard1
├── species_id = s0
├── name = ABTS
├── wavelength = 340.0
├── samples
│   ├── 0
│   │   └── Sample
│   │       ├── id = sample0
│   │       ├── concentration = 0.0
│   │       ├── conc_unit = umol / l
│   │       └── signal = 0.03998664610119449
│   ├── 1
│   │   └── Sample
│   │       ├── id = sample1
│   │       ├── concentration = 0.0
│   │       ├── conc_unit = umol / l
│   │       └── signal = 0.029908585548001833
│   ├── 2
│   │   └── Sample
│   │       ├── id = sample2
│   │       ├── concentration = 0.0
│   │       ├── conc_unit = umol / l
│   │       └── signal = 0.014567241538710035
│   ├── 3
│   │   └── Sample
│   │       ├── id = sample3
│   │       ├── concentration = 20.0
│   │       ├── conc_unit = umol / l
│   │       └── signal = 0.28800364244345167
│   ├── 4
│   │   └── Sample
│   │       ├── id = sample4
│   │       ├── concentration = 20.0
│   │       ├── conc_unit = umol / l
│   │       └── signal = 0.2753484688866737
│   ├── 5
│   │   

In [ ]:
cutoff = 1.3

concentrations = np.array([1, 2, 3, 4, 5]) * 10
signals = np.array([1, 2, 3, 4, 5]) / 2


concentrations = np.array([1, 2, 3, 4, 5]) * 10
signals = np.array([1, 2, 3, 4, 5]) / 2

above_cutoff_idx = [idx for idx,
                    signal in enumerate(signals) if signal < cutoff]

concentrations = concentrations[above_cutoff_idx]
signals = signals[above_cutoff_idx]

print(signals)

[0.5 1. ]
